In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
from  datetime import datetime, timedelta

# Data preprocessing
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
#Import DataFrames.
sales=pd.read_csv('sales_train_evaluation.csv')
#sales=reduce_mem_usage(sales)
#sales=pd.read_csv('sales_train_validation.csv')
#sales=reduce_mem_usage(sales)

In [5]:
price=pd.read_csv('sell_prices.csv')
price=reduce_mem_usage(price)

Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [6]:
calendar=pd.read_csv('calendar.csv')
calendar=reduce_mem_usage(calendar)

Mem. usage decreased to  0.12 Mb (41.9% reduction)


In [7]:
sample=pd.read_csv('sample_submission.csv')
sample=reduce_mem_usage(sample)

Mem. usage decreased to  2.09 Mb (84.5% reduction)


In [8]:
catcol = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
sales= pd.melt(sales, id_vars= catcol, 
               var_name='day', value_name='demand')
sales = reduce_mem_usage(sales)

Mem. usage decreased to 3273.49 Mb (9.4% reduction)


In [9]:
test2_rows = [row for row in sample['id'] if 'evaluation' in row]
test2 = sample[sample['id'].isin(test2_rows)]

test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                  'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

product = sales[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()


test2 = test2.merge(product, how = 'left', on = 'id')

test2 = pd.melt(test2, 
                id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                var_name = 'day', 
                value_name = 'demand')

sales['part'] = 'train'
test2['part'] = 'test'

In [10]:
data = pd.concat([sales,test2], axis = 0)
import gc
del sales,test2
gc.collect()

79

In [11]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train


In [ ]:
data.tail()

In [12]:
data.reset_index(drop=True,inplace=True)
data = data.loc[30000000:]
calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)

In [13]:
data = data.merge(price, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
del calendar,price,product
gc.collect()

40

In [14]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOUSEHOLD_1_331_WI_3_evaluation,HOUSEHOLD_1_331,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,0,train,2013-10-08,11337,NaN,NaN,NaN,NaN,1,0,1,4.878906
1,HOUSEHOLD_1_332_WI_3_evaluation,HOUSEHOLD_1_332,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,0,train,2013-10-08,11337,NaN,NaN,NaN,NaN,1,0,1,NaN
2,HOUSEHOLD_1_333_WI_3_evaluation,HOUSEHOLD_1_333,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,0,train,2013-10-08,11337,NaN,NaN,NaN,NaN,1,0,1,2.970703
3,HOUSEHOLD_1_334_WI_3_evaluation,HOUSEHOLD_1_334,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,11,train,2013-10-08,11337,NaN,NaN,NaN,NaN,1,0,1,0.979980
4,HOUSEHOLD_1_335_WI_3_evaluation,HOUSEHOLD_1_335,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,0,train,2013-10-08,11337,NaN,NaN,NaN,NaN,1,0,1,8.968750


In [15]:
data.dtypes

id               object
item_id          object
dept_id          object
cat_id           object
store_id         object
state_id         object
demand            int16
part             object
date             object
wm_yr_wk          int16
event_name_1     object
event_type_1     object
event_name_2     object
event_type_2     object
snap_CA            int8
snap_TX            int8
snap_WI            int8
sell_price      float16
dtype: object

In [16]:
from sklearn.preprocessing import LabelEncoder
nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    data[feature].fillna('unknown', inplace = True)

cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in cat:
    encoder = LabelEncoder()
    data[feature] = encoder.fit_transform(data[feature])

In [17]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOUSEHOLD_1_331_WI_3_evaluation,2326,5,2,9,2,0,train,2013-10-08,11337,30,4,2,2,1,0,1,4.878906
1,HOUSEHOLD_1_332_WI_3_evaluation,2327,5,2,9,2,0,train,2013-10-08,11337,30,4,2,2,1,0,1,NaN
2,HOUSEHOLD_1_333_WI_3_evaluation,2328,5,2,9,2,0,train,2013-10-08,11337,30,4,2,2,1,0,1,2.970703
3,HOUSEHOLD_1_334_WI_3_evaluation,2329,5,2,9,2,11,train,2013-10-08,11337,30,4,2,2,1,0,1,0.979980
4,HOUSEHOLD_1_335_WI_3_evaluation,2330,5,2,9,2,0,train,2013-10-08,11337,30,4,2,2,1,0,1,8.968750


In [18]:
data.dtypes

id               object
item_id           int32
dept_id           int32
cat_id            int32
store_id          int32
state_id          int32
demand            int16
part             object
date             object
wm_yr_wk          int16
event_name_1      int32
event_type_1      int32
event_name_2      int32
event_type_2      int32
snap_CA            int8
snap_TX            int8
snap_WI            int8
sell_price      float16
dtype: object

In [19]:
data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
data['rolling_mean_t7']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
data['rolling_std_t7']    = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
data['rolling_mean_t30']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())

In [20]:
data = reduce_mem_usage(data)

Mem. usage decreased to 1804.54 Mb (49.6% reduction)


In [21]:
data['rolling_mean_t90']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
data['rolling_std_t30']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
data['rolling_skew_t30']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
data['rolling_kurt_t30']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())

In [22]:
data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)

In [23]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['week'] = data['date'].dt.week
data['day'] = data['date'].dt.day
data['dayofweek'] = data['date'].dt.dayofweek

In [24]:
def weekend(arg):
    if arg==5 or arg==6:
        return 1
    else:
        return 0
data['isweekend'] = data['dayofweek'].apply(weekend)

In [25]:
data['revenue'] = data['demand'] * data['sell_price']
data['lag_revenue_t1'] = data.groupby(['id'])['revenue'].transform(lambda x: x.shift(28))
data['rolling_revenue_std_t28'] = data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).std())
data['rolling_revenue_mean_t28'] = data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).mean())
data.drop(['revenue'],axis=1,inplace=True)

In [26]:
features = ['item_id', 'cat_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30',
            'isweekend','lag_revenue_t1','rolling_revenue_std_t28','rolling_revenue_mean_t28'
        ]
data = reduce_mem_usage(data)

Mem. usage decreased to 2692.48 Mb (45.0% reduction)


In [27]:
x_train = data[data['part'] == 'train']
y_train = x_train['demand']
x_val = data[(data['date'] > '2016-04-24') & (data['part'] == 'train')]
y_val = x_val['demand']
test = data[data['date'] > '2016-04-24']
test.loc[test['part']=='train','id'] = test.loc[test['part']=='train','id'].str.replace('_evaluation','_validation')
x_train.drop(['demand','part',],inplace=True,axis=1)
x_val.drop(['demand','part',],inplace=True,axis=1)
test.drop(['demand','part',],inplace=True,axis=1)

In [28]:
del data
gc.collect()

52

In [29]:
from lightgbm import Dataset,train,plot_importance
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1}

In [30]:
category = x_train['dept_id'].unique()
category2 = x_train['store_id'].unique()
def dataset(categor,categor2):
    tindex = x_train[(x_train['dept_id']==categor) & (x_train['store_id']==categor2)].index.values
    vindex = x_val[(x_val['dept_id']==categor)&(x_val['store_id']==categor2)].index.values
    
    x_t,x_v,y_t,y_v,t =  x_train[(x_train['dept_id']==categor) & (x_train['store_id']==categor2)],
                        x_val[(x_val['dept_id']==categor)&(x_val['store_id']==categor2)],
                        y_train.loc[tindex], y_val.loc[vindex],
                        test[(test['dept_id']==categor) &(test['store_id']==categor2)]
    
    x_train.drop(tindex,axis=0,inplace=True)
    x_val.drop(vindex,axis=0,inplace=True)
    test.drop(test[(test['dept_id']==categor) &(test['store_id']==categor2)].index.values,axis=0,inplace=True)
    
    return x_t,x_v,y_t,y_v,t

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
TEST = None


for i in category:
    for j in category2:
        x_t,x_v,y_t,y_v,t = dataset(i,j)
        train_set = Dataset(x_t[features], y_t)
        val_set = Dataset(x_v[features], y_v)
        del x_t, y_t
        gc.collect()

        model = train(params, train_set, num_boost_round = 5000, early_stopping_rounds = 40, 
                      valid_sets = [train_set, val_set], verbose_eval = 1000)
        y_pred = model.predict(t[features])

        t['demand'] = y_pred

        TEST = pd.concat([TEST,t],axis=0)
        del x_v,y_v,t,train_set,val_set,y_pred,model
        gc.collect()

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.084502 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4115
[LightGBM] [Info] Number of data points in the train set: 509332, number of used features: 33
[LightGBM] [Info] Start training from score 1.040231
Training until validation scores don't improve for 40 rounds
[1000]	training's rmse: 1.19306	valid_1's rmse: 1.26201
[2000]	training's rmse: 1.08074	valid_1's rmse: 1.11463
[3000]	training's rmse: 1.00408	valid_1's rmse: 1.03622
[4000]	training's rmse: 0.944439	valid_1's rmse: 0.975225
[5000]	training's rmse: 0.896005	valid_1's rmse: 0.925215
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 0.896005	valid_1's rmse: 0.925215
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.087053 seconds.
You can set `force_row_wise=true

[2000]	training's rmse: 1.35714	valid_1's rmse: 1.36388
[3000]	training's rmse: 1.24842	valid_1's rmse: 1.2588
[4000]	training's rmse: 1.1692	valid_1's rmse: 1.18511
[5000]	training's rmse: 1.10416	valid_1's rmse: 1.11677
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 1.10416	valid_1's rmse: 1.11677
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059074 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3611
[LightGBM] [Info] Number of data points in the train set: 493370, number of used features: 33
[LightGBM] [Info] Start training from score 0.225537
Training until validation scores don't improve for 40 rounds
[1000]	training's rmse: 0.475181	valid_1's rmse: 0.542332
[2000]	training's rmse: 0.446482	valid_1's rmse: 0.502247
[3000]	training's rmse: 0.425893	valid_1's rmse: 0.472902
[4000]	training's rmse: 0.408

[1000]	training's rmse: 0.492866	valid_1's rmse: 0.523162
[2000]	training's rmse: 0.460643	valid_1's rmse: 0.487005
[3000]	training's rmse: 0.436343	valid_1's rmse: 0.457979
[4000]	training's rmse: 0.41778	valid_1's rmse: 0.436976
[5000]	training's rmse: 0.40212	valid_1's rmse: 0.419797
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 0.40212	valid_1's rmse: 0.419797
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024843 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3988
[LightGBM] [Info] Number of data points in the train set: 206928, number of used features: 33
[LightGBM] [Info] Start training from score 0.982491
Training until validation scores don't improve for 40 rounds
[1000]	training's rmse: 1.12664	valid_1's rmse: 1.24097
[2000]	training's rmse: 0.977942	valid_1's rmse: 1.07736
[3000]	training's rmse:

[1000]	training's rmse: 1.48731	valid_1's rmse: 1.69604
[2000]	training's rmse: 1.25646	valid_1's rmse: 1.45606
[3000]	training's rmse: 1.11884	valid_1's rmse: 1.29493
[4000]	training's rmse: 1.0179	valid_1's rmse: 1.16964
[5000]	training's rmse: 0.938146	valid_1's rmse: 1.0693
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 0.938146	valid_1's rmse: 1.0693
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045453 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4127
[LightGBM] [Info] Number of data points in the train set: 381284, number of used features: 33
[LightGBM] [Info] Start training from score 0.985336
Training until validation scores don't improve for 40 rounds
[1000]	training's rmse: 1.18913	valid_1's rmse: 1.36017
[2000]	training's rmse: 1.07589	valid_1's rmse: 1.22983
[3000]	training's rmse: 0.998099	v

In [ ]:
predictions1 = TEST[TEST['id'].apply(lambda x: "validation" in x)][['id', 'date', 'sales']]
predictions2 = TEST[TEST['id'].apply(lambda x: "evaluation" in x)][['id', 'date', 'sales']]

In [ ]:
prediction1 = pd.pivot(predictions1, index = 'id', columns = 'date', values = 'sales').reset_index()
prediction2 = pd.pivot(predictions2, index = 'id', columns = 'date', values = 'sales').reset_index()
prediction1.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
prediction2.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [ ]:
predictions = pd.concat([prediction1,prediction2],axis=0)
predictions.to_csv("submissionstoreanddeptwise.csv",index=False)

In [ ]:
sub=pd.read_csv("submissionstoreanddeptwise.csv")
sub.head()

In [ ]:
sub.info()

In [ ]:
sub.dtypes

In [ ]:
!gzip submissionstoreanddeptwise.csv